<a href="https://www.kaggle.com/code/deepkayastha6890/bank-loan-risk-pattern-finder?scriptVersionId=295769260" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [22]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os

# Addition paths
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/bank-transaction-data/bank.xlsx


In [23]:
file_path = 'bank.xlsx'

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "apoorvwatsky/bank-transaction-data",
    file_path,
)

# Dataframe first five rows
print("\n\nFirst 5 records:")
display(df.head())


/tmp/ipykernel_55/3806805928.py:3: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(




First 5 records:


,Account No,DATE,TRANSACTION DETAILS,CHQ.NO.,VALUE DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,.
0,409000611074',2017-06-29,TRF FROM Indiaforensic SERVICES,NaN,2017-06-29,NaN,1000000.0,1000000.0,.
1,409000611074',2017-07-05,TRF FROM Indiaforensic SERVICES,NaN,2017-07-05,NaN,1000000.0,2000000.0,.
2,409000611074',2017-07-18,FDRL/INTERNAL FUND TRANSFE,NaN,2017-07-18,NaN,500000.0,2500000.0,.
3,409000611074',2017-08-01,TRF FRM Indiaforensic SERVICES,NaN,2017-08-01,NaN,3000000.0,5500000.0,.
4,409000611074',2017-08-16,FDRL/INTERNAL FUND TRANSFE,NaN,2017-08-16,NaN,500000.0,6000000.0,.


In [24]:
def DataCleaning(df): 
    
    cleaningData = df.copy()

    #Finding the information 
    print("\n\nInformations:\n",cleaningData.info())

    # Finding the null values
    majorColumns = ['DEPOSIT AMT','WITHDRAWAL AMT','DATE','VALUE DATE']
    for i in majorColumns:
        print(f'\n\n{i} : ',cleaningData[i].isnull().sum())

    # Filling the null values
    nullValuesFiller = ['DEPOSIT AMT','WITHDRAWAL AMT','CHQ.NO.']
    for i in nullValuesFiller:
        cleaningData[i] = cleaningData[i].fillna(0)

    # Just changing the date name
    cleaningData['ActualDate'] = cleaningData['VALUE DATE']
    
    # Seperating the day , month , year 
    cleaningData['ActualDay'] = cleaningData['VALUE DATE'].dt.day 
    cleaningData['ActualMonth'] = cleaningData['VALUE DATE'].dt.month
    cleaningData['ActualYear'] = cleaningData['VALUE DATE'].dt.year

    # Sorting the data from starting date
    cleaningData = cleaningData.sort_values(by="VALUE DATE", ascending=False)

    # Droping unuseful data colums [Account No, , . ,CHQ.NO. ]
    cleaningData = cleaningData.drop(columns=['.','CHQ.NO.','VALUE DATE'])

    # Finding the columns 
    print('\n')
    print(cleaningData.columns)

    # finding the shape
    print("\nDataset Size",cleaningData.shape)

    # Finding the data discribtion 
    print('\nDiscribtion of the data : ')
    display(cleaningData.describe())
    
    return cleaningData
    
cleanData = DataCleaning(df)
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116201 entries, 0 to 116200
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Account No           116201 non-null  object        
 1   DATE                 116201 non-null  datetime64[ns]
 2   TRANSACTION DETAILS  113702 non-null  object        
 3   CHQ.NO.              905 non-null     float64       
 4   VALUE DATE           116201 non-null  datetime64[ns]
 5   WITHDRAWAL AMT       53549 non-null   float64       
 6   DEPOSIT AMT          62652 non-null   float64       
 7   BALANCE AMT          116201 non-null  float64       
 8   .                    116201 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(3)
memory usage: 8.0+ MB


Informations:
 None


DEPOSIT AMT :  53549


WITHDRAWAL AMT :  62652


DATE :  0


VALUE DATE :  0


Index(['Account No', 'DATE', 'TRANSACTION DETAILS', 'WITHDRAWAL AMT',
       '

,DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,ActualDate,ActualDay,ActualMonth,ActualYear
count,116201,1.162010e+05,1.162010e+05,1.162010e+05,116201,116201.000000,116201.000000,116201.000000
mean,2017-05-20 00:08:40.477448448,2.068757e+06,2.052394e+06,-1.404852e+09,2017-05-20 00:04:43.288440064,15.615907,6.680313,2016.869993
min,2015-01-01 00:00:00,0.000000e+00,0.000000e+00,-2.045201e+09,2015-01-01 00:00:00,1.000000,1.000000,2015.000000
25%,2016-05-30 00:00:00,0.000000e+00,0.000000e+00,-1.690383e+09,2016-05-30 00:00:00,8.000000,4.000000,2016.000000
50%,2017-06-05 00:00:00,0.000000e+00,5.000000e+03,-1.661395e+09,2017-06-05 00:00:00,15.000000,7.000000,2017.000000
75%,2018-05-26 00:00:00,2.682500e+04,5.000000e+05,-1.236888e+09,2018-05-26 00:00:00,23.000000,10.000000,2018.000000
max,2019-03-05 00:00:00,4.594475e+08,5.448000e+08,8.500000e+06,2019-03-05 00:00:00,31.000000,12.000000,2019.000000
std,NaN,7.696881e+06,6.652164e+06,5.348202e+08,NaN,8.602524,3.491998,1.131124


In [25]:
def DataDiscribtion(df):

    dataDiscribe = df.copy()
    
    # Year wise count
    total_year_counts = dataDiscribe['ActualYear'].value_counts()

    print("Total list of year : ")
    display(total_year_counts.head())
    print("\n")
    
    # function wise distributions
    display(dataDiscribe.describe())

    # Less Then Zero balance
    count =0
    for value in dataDiscribe['BALANCE AMT']:
        if(value < 0):
            count = count +1;
    
    # Counting the minus value
    print("\n\nMinus Value in balance : ",count)
    
    # Finding the data distribution
    print('\n\nUnique value of the account numbers')
    display(dataDiscribe['Account No'].unique())
    
DataDiscribtion(cleanData)


Total list of year : 


ActualYear
2018    35534
2016    30373
2017    29112
2015    15658
2019     5524
Name: count, dtype: int64

,DATE,WITHDRAWAL AMT,DEPOSIT AMT,BALANCE AMT,ActualDate,ActualDay,ActualMonth,ActualYear
count,116201,1.162010e+05,1.162010e+05,1.162010e+05,116201,116201.000000,116201.000000,116201.000000
mean,2017-05-20 00:08:40.477448448,2.068757e+06,2.052394e+06,-1.404852e+09,2017-05-20 00:04:43.288440064,15.615907,6.680313,2016.869993
min,2015-01-01 00:00:00,0.000000e+00,0.000000e+00,-2.045201e+09,2015-01-01 00:00:00,1.000000,1.000000,2015.000000
25%,2016-05-30 00:00:00,0.000000e+00,0.000000e+00,-1.690383e+09,2016-05-30 00:00:00,8.000000,4.000000,2016.000000
50%,2017-06-05 00:00:00,0.000000e+00,5.000000e+03,-1.661395e+09,2017-06-05 00:00:00,15.000000,7.000000,2017.000000
75%,2018-05-26 00:00:00,2.682500e+04,5.000000e+05,-1.236888e+09,2018-05-26 00:00:00,23.000000,10.000000,2018.000000
max,2019-03-05 00:00:00,4.594475e+08,5.448000e+08,8.500000e+06,2019-03-05 00:00:00,31.000000,12.000000,2019.000000
std,NaN,7.696881e+06,6.652164e+06,5.348202e+08,NaN,8.602524,3.491998,1.131124




Minus Value in balance :  113276


Unique value of the account numbers


array(["409000362497'", "409000438620'", "1196428'", "409000438611'",
       "409000493210'", "409000493201'", "409000405747'", "409000425051'",
       "1196711'", "409000611074'"], dtype=object)

In [50]:
# Count of compressed features
def groupByCounting(df, respectedMainFeature, copresstionFeatures):
    return (
        df
        .groupby(respectedMainFeature)[copresstionFeatures]
        .count()
        .sort_index()
    )
# Sum of the compress features
def groupBySum(df, respectedMainFeature, copresstionFeatures):
    return (
        df
        .groupby(respectedMainFeature)[copresstionFeatures]
        .sum()
        .sort_index()
    )

# Day wise data process and account No wise classifier
def build_daily_account_log(df):

    # Day wise agrigated data and this data is days wise
    daily_log = (
        df
        .sort_values(['Account No', 'DATE'])
        .groupby(['Account No', 'DATE'])
        .agg(
            TotalDeposit=('DEPOSIT AMT', 'sum'),
            TotalWithdrawal=('WITHDRAWAL AMT', 'sum'),
            EndBalance=('BALANCE AMT', 'last'),
            TxnCount=('BALANCE AMT', 'count')
        )
        .reset_index()
    )

    # Net movement for the day
    daily_log['NetChange'] = (
        daily_log['TotalDeposit'] - daily_log['TotalWithdrawal']
    )
    
    # If overdraf this increase
    daily_account_df['IsOverdraft'] = daily_account_df['EndBalance'] < 0
    
    # Marking overdraft
    daily_account_df['OverdraftAmount'] = (
        -daily_account_df['EndBalance']
    ).clip(lower=0)

    # Daily balance change
    daily_account_df['DailyBalanceChange'] = (
    daily_account_df
        .groupby('Account No')['EndBalance']
        .diff()
    )
    daily_account_df['MONTH'] = daily_account_df['DATE'].dt.to_period('M')

    # Two dataframes are there
    return daily_log , daily_account_df
_ , manngedData = build_daily_account_log(cleanData)

display(manngedData.shape)
display(manngedData.head())

def monthly_account_features(df):

    df = df.sort_values(['Account No', 'DATE']).copy()

    # STREAK LOGIC (for Recovery Velocity)
    df['StreakStart'] = (
        (df['IsOverdraft']) &
        (~df.groupby('Account No')['IsOverdraft'].shift(fill_value=False))
    )

    df['StreakID'] = df.groupby('Account No')['StreakStart'].cumsum()

    overdraft_days = df[df['IsOverdraft']]

    streaks = (
        overdraft_days
        .groupby(['Account No', 'MONTH', 'StreakID'])
        .size()
        .reset_index(name='StreakLength')
    )

    # Avg overdraft streak per month (Recovery Velocity)
    recovery_velocity = (
        streaks
        .groupby(['Account No', 'MONTH'])['StreakLength']
        .mean()
        .reset_index(name='RecoveryVelocity')
    )

    # MONTHLY CORE AGGREGATIONS
    monthly = (
        df
        .groupby(['Account No', 'MONTH'])
        .agg(
            DaysInMonth=('DATE', 'nunique'),
            StressDays=('IsOverdraft', 'sum'),
            NetCashflow=('NetChange', 'sum'),
            TotalTxn=('TxnCount', 'sum'),
            MaxWithdrawal=('TotalWithdrawal', 'max'),
            AvgEndBalance=('EndBalance', 'mean'),
            BalanceVolatility=('EndBalance', 'std')
        )
        .reset_index()
    )

    # 2. Whale Impact
    monthly['WhaleImpact'] = (
        monthly['MaxWithdrawal'] /
        monthly['AvgEndBalance'].abs()
    )

    # 3. Stress Duration
    monthly['StressDuration'] = (
        monthly['StressDays'] / monthly['DaysInMonth']
    )

    # 5. Operational Intensity
    monthly['OperationalIntensity'] = (
        monthly['TotalTxn'] / monthly['DaysInMonth']
    )

    # MERGE RECOVERY VELOCITY
    final_features = monthly.merge(
        recovery_velocity,
        on=['Account No', 'MONTH'],
        how='left'
    )

    # If no overdraft streaks → velocity = 0
    final_features['RecoveryVelocity'] = (
        final_features['RecoveryVelocity'].fillna(0)
    )

    return final_features
monthly_features = monthly_account_features(manngedData)
monthly_features.head()



(6218, 11)

,Account No,DATE,TotalDeposit,TotalWithdrawal,EndBalance,TxnCount,NetChange,IsOverdraft,OverdraftAmount,DailyBalanceChange,MONTH
0,1196428',2015-01-01,2000000.0,0.00,-1.584916e+09,2,2000000.00,True,1.584916e+09,NaN,2015-01
1,1196428',2015-01-02,2462620.0,5500056.18,-1.587953e+09,9,-3037436.18,True,1.587953e+09,-3037436.18,2015-01
2,1196428',2015-01-03,2948160.0,2200000.00,-1.587205e+09,4,748160.00,True,1.587205e+09,748160.00,2015-01
3,1196428',2015-01-05,3877320.0,3231000.00,-1.586736e+09,5,646320.00,True,1.586736e+09,469000.00,2015-01
4,1196428',2015-01-06,3650750.0,3880056.18,-1.587131e+09,6,-229306.18,True,1.587131e+09,-395236.18,2015-01


,Account No,MONTH,DaysInMonth,StressDays,NetCashflow,TotalTxn,MaxWithdrawal,AvgEndBalance,BalanceVolatility,WhaleImpact,StressDuration,OperationalIntensity,RecoveryVelocity
0,1196428',2015-01,26,26,2409931.64,191,9230000.0,-1.586814e+09,2.355750e+06,0.005817,1.0,7.346154,26.0
1,1196428',2015-02,24,24,1455434.00,262,9350500.0,-1.586019e+09,2.706476e+06,0.005896,1.0,10.916667,24.0
2,1196428',2015-03,25,25,-4449586.00,237,9750000.0,-1.586510e+09,2.486767e+06,0.006146,1.0,9.480000,25.0
3,1196428',2015-04,24,24,489124.00,256,10000000.0,-1.587227e+09,2.138390e+06,0.006300,1.0,10.666667,24.0
4,1196428',2015-05,26,26,590534.00,378,14500000.0,-1.586181e+09,2.770573e+06,0.009141,1.0,14.538462,26.0
